<a href="https://colab.research.google.com/github/Param-Bhatt/NNFL-Assignment/blob/master/2/Q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h3>Navigating to respective directory

In [2]:
%cd "/content/drive/My Drive/NNFL-Assignments/2"
!ls

/content/drive/My Drive/NNFL-Assignments/2
assignment2.gdoc  class2_images    class_label.mat  data5.mat	  input.mat
assignment2.pdf   class3_images    data55.xlsx	    input_a2.mat  label.mat
class1_images	  class_label.csv  data5.csv	    input.gsheet  Q8.ipynb


<h3> Importing all the libraries required

In [3]:
from __future__ import division, print_function
import scipy.io
import numpy as np
import random as rnd
import pandas as pd
mat = scipy.io.loadmat('data5.mat')
x = mat['x']
x = np.asarray(x)
np.random.shuffle(x)
x.shape

len = x.shape[0]
train_len = int(0.7 * len)
validate_len = int((0.1 * len) + 1)
test_len = int((0.2 * len) + 1)

train_data = x[0:train_len,:]
validate_data = x[train_len+1:train_len+1+validate_len,:]
test_data = x[train_len+validate_len+1:,:]

x_train = train_data[:,0:72]
y_train = train_data[:,-1]
x_validate = validate_data[:,0:72]
y_validate = validate_data[:,-1]
x_test = test_data[:,0:72]
y_test = test_data[:,-1]

<h3> Extracting the required data </h3>

In [4]:
mat = scipy.io.loadmat('data5.mat')
data = mat['x']
#transform to numpy array
data = np.asarray(data)
#shuffling the data for random choices
np.random.shuffle(data)

<h4> Dump the data to a csv file (optional) </h4>

In [5]:
#dump the data to csv
np.savetxt("data5.csv", data, delimiter=",")

<h3>Form datasets for use, and sanitize them as needed </h3>

In [6]:
len = data.shape[0]
train_len = int(0.7 * len)
validate_len = int((0.1 * len) + 1)
test_len = int((0.2 * len) + 1)
#dividing the data
train_data = data[0:train_len,:]
validate_data = data[train_len+1:train_len+1+validate_len,:]
test_data = data[train_len+validate_len+1:,:]
#making train,validate and test data
x_train = train_data[:,0:72]
y_train = train_data[:,-1]
x_validate = validate_data[:,0:72]
y_validate = validate_data[:,-1]
x_test = test_data[:,0:72]
y_test = test_data[:,-1]
#convert the classes to 1 & -1
for i in range(y_train.shape[0]):
  y_train[i] = int(y_train[i])
  if int(y_train[i]) == 0:
    y_train[i] = -1
for i in range(y_validate.shape[0]):
  y_validate[i] = int(y_validate[i])
  if int(y_validate[i]) == 0:
    y_validate[i] = -1
for i in range(y_test.shape[0]):
  y_test[i] = int(y_test[i])
  if int(y_test[i]) == 0:
    y_test[i] = -1


<h3> The SVM class </h3>

In [7]:
class SVM():
    """
        Simple implementation of a Support Vector Machine using the
        Sequential Minimal Optimization (SMO) algorithm for training.
    """
    def __init__(self, max_iter=10000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
    def fit(self, X, y):
        # Initialization
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                i = self.get_rnd_int(0, n-1, j) # Get random int i~=j
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                # Compute E_i, E_j
                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                # Set new alpha values
                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            # Check convergence
            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if count >= self.max_iter:
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return
        # Compute final model parameters
        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)
        # Get support vectors
        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = X[alpha_idx, :]
        return support_vectors, count
    def predict(self, X):
        return self.h(X, self.w, self.b)
    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)
    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))
    # Prediction
    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)
    # Prediction error
    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k
    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, y_j, y_i):
        if(y_i != y_j):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
    def get_rnd_int(self, a,b,z):
        i = z
        cnt=0
        while i == z and cnt<1000:
            i = rnd.randint(a,b)
            cnt=cnt+1
        return i
    # Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)

<h3> Function to calculate accuracies </h3>

In [8]:
def class_acc(y_pred,y_actual):
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  print(confmat)
  Acc = (confmat[0][0] + confmat[1][1])/sum(sum(confmat))
  Acc1 = confmat[0][0]/sum(confmat[0])
  Acc2 = confmat[1][1]/sum(confmat[1])
  return Acc,Acc1,Acc2

<h1><b><i><u>RUN

In [10]:
%%time

model = SVM(max_iter=10000, kernel_type='linear', C=1.0, epsilon=0.001)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
Acc,Acc1,Acc2=class_acc(y_test,y_pred)
print('Overall Class Accuracy:',Acc)
print('Class 1 Accuracy:',Acc1)
print('Class 2 Accuracy:',Acc2)


[[181  36]
 [ 26 186]]
Overall Class Accuracy: 0.8554778554778555
Class 1 Accuracy: 0.8341013824884793
Class 2 Accuracy: 0.8773584905660378
CPU times: user 951 ms, sys: 731 ms, total: 1.68 s
Wall time: 878 ms
